# Create a zarr with masks for an idr0041 image

Download idr0041 image and corresponding mask, for example:
- `omero-ms-zarr/src/scripts/omero_to_zarr.py 3430725` ([Image](https://idr.openmicroscopy.org/webclient/?show=image-3430725))
- `omero-ms-zarr/src/scripts/omero_to_zarr.py 3430805` ([Mask](https://idr.openmicroscopy.org/webclient/?show=image-3430805))

[Open in OMERO.iviewer](https://idr.openmicroscopy.org/iviewer/?images=3430725,3430805)

In [1]:
import numpy as np
import requests
import zarr

In [2]:
image = zarr.open('3430725.zarr', mode='r')
mask = zarr.open('3430805.zarr', mode='r')
print(f'3430725.zarr: {image[0].shape}')
print(f'3430805.zarr: {mask[0].shape}')
assert image[0].shape[2:] == mask[0].shape[2:]

3430725.zarr: (1, 3, 31, 256, 256)
3430805.zarr: (1, 2, 31, 256, 256)


Get the mask image metadata as this will be used to name the masks in the Zarr (though as with images the name can be anything)

In [3]:
mask_info = requests.get('https://idr.openmicroscopy.org/iviewer/image_data/3430806/').json()
mask_names = [c['label'] for c in mask_info['channels']]
assert len(mask_names) == mask[0].shape[1]

Copy the image to `output.zarr`. Save the two binary masks using the channel labels from the idr `NUC` and `CELL`.
The mask arrays have the same number of dimensions as the image `T, C, Z, Y, X` but some dimensions are singular.

In [4]:
output = zarr.open('output.zarr')
output[0] = image[0]
out_masks = output.create_group('masks', overwrite=True)

for c in range(mask[0].shape[1]):
    print(f'Mask: {mask_names[c]}')
    m = out_masks.create_dataset(mask_names[c], shape=(1, 1) + mask[0].shape[2:], dtype=np.bool)
    m[0, 0, :] = mask[0][0, c, :, :, :]

Mask: NUC
Mask: CELL


Set attributes on the masks:
- The parent image (the name of the Zarr image array)
- The source plane(s) of each mask used to calculate the mask for any singular dimensions.
Unlike OMERO these are arrays to indicate a mask can be calculated using information from more than one plane.
In this example this is obtained by looking at the images, and figuring out which channel(s) were used to create the segmentation.

The type of the mask can be inferred from the `dtype` of the array:
- `bool`: binary bitmask
- `integer`: label image

In [5]:
output.masks.NUC.attrs['image'] = {
    'array': '0',
    'source': {
#         'ts': [],
        'cs': [1],  # Channel NUC
#         'zs': [],
#         'ys': [],
#         'xs': [],
    },
}
output.masks.CELL.attrs['image'] = {
    'array': '0',
    'source': {
#         'ts': [],
        'cs': [0],  # Channel POI
#         'zs': [],
#         'ys': [],
#         'xs': [],
    },
}